In [2]:
import os
import preprocessing

from gensim import corpora

In [29]:
import preprocessing
import gensim


def read_corpus(metadata_path='data/arxiv_metadata.json', tokens_only=False):
    arx = preprocessing.flat_unique(path=metadata_path)
    for i, paper in enumerate(arx):
            if tokens_only:
                yield gensim.utils.simple_preprocess(paper['title'] + paper['summary'])
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(
                    gensim.utils.simple_preprocess(paper['title'] + paper['summary']), [i])
    
    
#test = list(read_corpus(tokens_only=True))
train_corpus = list(read_corpus(tokens_only=False))

In [91]:
import os

def texts_corpus(textdir='data/texts/'):
    files = os.listdir(textdir)
    for file in files:
        if '.txt' not in file:
            continue
        with open(textdir + file) as f:
            t = f.read()
            #yield [word for word in t.replace('\n', ' ').split() if len(word) > 3 and word.isalpha()]
            yield gensim.utils.simple_preprocess(t.replace('\n', ' '))
            
train_corpus = texts_corpus()

In [92]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=100)
model.build_vocab(train_corpus) # ?

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

AttributeError: 'list' object has no attribute 'words'

In [57]:
import random

papers = preprocessing.flat_unique()


"""
# Get summaries
train_summaries = [paper['summary'] for paper in papers]
train_titles = [paper['title'] for paper in papers]
train_titles = [paper['title'] for paper in papers]
"""
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_summaries) - 1)
inferred_vector = model.infer_vector(train_summaries[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(train_titles[doc_id], train_summaries[doc_id]))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], train_summaries[sims[index][0]]))

Test Document (Statistical Basis for Predicting Technological Progress): «Forecasting technological progress is of great interest to engineers, policy
makers, and private investors. Several models have been proposed for predicting
technological improvement, but how well do these models perform? An early
hypothesis made by Theodore Wright in 1936 is that cost decreases as a power
law of cumulative production. An alternative hypothesis is Moore's law, which
can be generalized to say that technologies improve exponentially with time.
Other alternatives were proposed by Goddard, Sinclair et al., and Nordhaus.
These hypotheses have not previously been rigorously tested. Using a new
database on the cost and production of 62 different technologies, which is the
most expansive of its kind, we test the ability of six different postulated
laws to predict future costs. Our approach involves hindcasting and developing
a statistical model to rank the performance of the postulated laws. Wright's
law

In [58]:
arx = preprocessing.flat_unique()
for i in range(len(arx)):
    terms = [item['term'] for item in arx[i]['tags'] if 'q-fin' in item['term']][0]     
    #print(terms)

In [65]:
def add_edges(edges, paper):
    authors = paper['authors']
    for author in authors:
        for other in authors.remove(author):
            if other not in edges[author]:
                pass